In [17]:
from binance.client import Client
from dotenv import load_dotenv
import os

# Step 1: Load environment variables from .env file
load_dotenv()

# Step 2: Access the API keys
api_key = os.getenv('BINANCE_API_KEY')
api_secret = os.getenv('BINANCE_SECRET_KEY')

# Step 3: Initialize the Binance client
client = Client(api_key, api_secret)

# Step 4: Fetch live market data
def get_live_market_data(symbol):
    # Get the latest price for a symbol (e.g., BTC/USDT)
    ticker = client.get_symbol_ticker(symbol=symbol)
    print(f"Current price of {symbol}: {ticker['price']}")

    # Get historical klines (candlestick data)
    klines = client.get_historical_klines(symbol, Client.KLINE_INTERVAL_1HOUR, "1 day ago UTC")
    for kline in klines:
        print(kline)

# Example usage
symbol = 'SOLUSDT'
get_live_market_data(symbol)

Current price of SOLUSDT: 131.06000000
[1742670000000, '129.74000000', '130.27000000', '129.62000000', '129.97000000', '92268.58900000', 1742673599999, '11990787.41126000', 24952, '42331.83100000', '5500297.94268000', '0']
[1742673600000, '129.97000000', '130.06000000', '129.10000000', '129.12000000', '52936.21200000', 1742677199999, '6857689.98275000', 24441, '23265.47300000', '3015661.79323000', '0']
[1742677200000, '129.12000000', '129.69000000', '128.32000000', '128.55000000', '48953.27600000', 1742680799999, '6311609.61020000', 25891, '20226.81900000', '2607938.92411000', '0']
[1742680800000, '128.55000000', '128.85000000', '128.34000000', '128.74000000', '34031.16500000', 1742684399999, '4377797.37138000', 21338, '17286.84400000', '2224077.18635000', '0']
[1742684400000, '128.75000000', '128.77000000', '128.12000000', '128.51000000', '36780.59200000', 1742687999999, '4720222.28159000', 23235, '16179.44200000', '2076349.06437000', '0']
[1742688000000, '128.50000000', '128.89000000

In [25]:
# Import required libraries
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objs as go
from binance.client import Client
from dotenv import load_dotenv
import os
import numpy as np
from scipy.integrate import solve_ivp
import time
import threading

# Load environment variables from .env file
load_dotenv()

# Initialize Binance client
api_key = os.getenv('BINANCE_API_KEY')
api_secret = os.getenv('BINANCE_SECRET_KEY')
client = Client(api_key, api_secret)

# Define the symbol for live data (e.g., BTCUSDT)
symbol = 'SOLUSDT'

# Monte Carlo simulation function
def monte_carlo_simulation(live_price, n_simulations=1000, n_days=252, mu=0.0001, sigma=0.01):
    """
    Generate Monte Carlo price paths using Geometric Brownian Motion (GBM).
    """
    dt = 1 / n_days
    paths = np.zeros((n_days, n_simulations))
    paths[0] = live_price

    for t in range(1, n_days):
        rand = np.random.normal(0, 1, n_simulations)
        paths[t] = paths[t - 1] * np.exp((mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * rand)

    return paths

# Lorenz attractor function
def lorenz_system(t, state, sigma, rho, beta):
    """
    Define the Lorenz system of differential equations.
    """
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x * (rho - z) - y
    dzdt = x * y - beta * z
    return [dxdt, dydt, dzdt]

# Parameters for the Lorenz system
sigma = 10
rho = 28
beta = 8 / 3

# Initial conditions for the Lorenz system
initial_state = [1, 1, 1]

# Time span for solving the Lorenz system
t_span = (0, 50)
t_eval = np.linspace(t_span[0], t_span[1], 10000)

# Solve the Lorenz system
solution = solve_ivp(lorenz_system, t_span, initial_state, args=(sigma, rho, beta), t_eval=t_eval)

# Extract the x-component of the Lorenz attractor
lorenz_x = solution.y[0]

# Scale the Lorenz x-component to match the price range
def scale_lorenz(lorenz_x, price_paths):
    """
    Scale the Lorenz attractor's x-component to match the price range.
    """
    scaled_lorenz = (lorenz_x - np.mean(lorenz_x)) / np.std(lorenz_x)
    scaled_lorenz = scaled_lorenz * volatility
    return scaled_lorenz

# Initialize Dash app
app = Dash(__name__)

# Layout of the dashboard
app.layout = html.Div([
    html.H1(f"Hybrid Monte Carlo-Chaos Attractor Dashboard ({symbol})"),
    dcc.Graph(id='live-graph'),
    dcc.Interval(id='interval-component', interval=60*1000, n_intervals=0),  # Update every 60 seconds
    html.Div(id='suggestions')
])

# Global variable to store the latest data
latest_data = {'price': None, 'paths': None, 'scaled_lorenz': None}

# Function to fetch live data and update the model
def update_data():
    while True:
        try:
            # Fetch live price data from Binance
            live_price = float(client.get_symbol_ticker(symbol=symbol)['price'])
            print(f"Live price of {symbol}: {live_price}")

            # Generate Monte Carlo price paths
            price_paths = monte_carlo_simulation(live_price)

            # Scale the Lorenz attractor's x-component
            scaled_lorenz = scale_lorenz(lorenz_x, volatility=5)###Adjust as needed

            # Update the global variable
            latest_data['price'] = live_price
            latest_data['paths'] = price_paths
            latest_data['scaled_lorenz'] = scaled_lorenz

        except Exception as e:
            print(f"Error: {e}")

        # Wait for 60 seconds before the next update
        time.sleep(60)

# Start the data update thread
thread = threading.Thread(target=update_data)
thread.daemon = True
thread.start()

# Callback to update the graph and suggestions
@app.callback(
    [Output('live-graph', 'figure'),
     Output('suggestions', 'children')],
    [Input('interval-component', 'n_intervals')]
)
def update_graph(n):
    # Get the latest data
    live_price = latest_data['price']
    price_paths = latest_data['paths']
    scaled_lorenz = latest_data['scaled_lorenz']

    if live_price is None or price_paths is None or scaled_lorenz is None:
        return go.Figure(), "Waiting for data..."

    # Combine Monte Carlo paths with chaos attractor dynamics
    hybrid_paths = price_paths * scaled_lorenz[:, np.newaxis]

    # Create the graph
    fig = go.Figure()
    for i in range(hybrid_paths.shape[1]):
        fig.add_trace(go.Scatter(
            y=hybrid_paths[:, i],
            mode='lines',
            line=dict(width=1),
            name=f'Path {i+1}'
        ))
    fig.update_layout(
        title=f"Hybrid Monte Carlo-Chaos Attractor Price Paths (Live Data - {symbol})",
        xaxis_title="Time (Days)",
        yaxis_title="Price"
    )

    # Generate suggestions based on the latest price
    if live_price > np.mean(hybrid_paths[-1]):
        suggestion = "The current price is above the average predicted price. Consider taking profits or reducing exposure."
    else:
        suggestion = "The current price is below the average predicted price. Consider buying or increasing exposure."

    return fig, html.Div([
        html.H3("Trading Suggestion:"),
        html.P(suggestion)
    ])

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)

ObsoleteAttributeException: app.run_server has been replaced by app.run

In [27]:
# Import required libraries
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objs as go
from binance.client import Client
from dotenv import load_dotenv
import os
import numpy as np
from scipy.integrate import solve_ivp
import time
import threading

# Load environment variables from .env file
load_dotenv()

# Initialize Binance client
api_key = os.getenv('BINANCE_API_KEY')
api_secret = os.getenv('BINANCE_SECRET_KEY')
client = Client(api_key, api_secret)

# Define the symbol for live data (e.g., SOLUSDT)
symbol = 'SOLUSDT'

# Monte Carlo simulation function
def monte_carlo_simulation(live_price, n_simulations=1000, n_days=252, mu=0.0001, sigma=0.01):
    """
    Generate Monte Carlo price paths using Geometric Brownian Motion (GBM).
    """
    dt = 1 / n_days
    paths = np.zeros((n_days, n_simulations))
    paths[0] = live_price

    for t in range(1, n_days):
        rand = np.random.normal(0, 1, n_simulations)
        paths[t] = paths[t - 1] * np.exp((mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * rand)

    return paths

# Lorenz attractor function
def lorenz_system(t, state, sigma, rho, beta):
    """
    Define the Lorenz system of differential equations.
    """
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x * (rho - z) - y
    dzdt = x * y - beta * z
    return [dxdt, dydt, dzdt]

# Parameters for the Lorenz system
sigma = 10
rho = 28
beta = 8 / 3

# Initial conditions for the Lorenz system
initial_state = [1, 1, 1]

# Time span for solving the Lorenz system
t_span = (0, 50)
t_eval = np.linspace(t_span[0], t_span[1], 10000)

# Solve the Lorenz system
solution = solve_ivp(lorenz_system, t_span, initial_state, args=(sigma, rho, beta), t_eval=t_eval)

# Extract the x-component of the Lorenz attractor
lorenz_x = solution.y[0]

# Scale the Lorenz x-component to match the price range
def scale_lorenz(lorenz_x, price_paths):
    """
    Scale the Lorenz attractor's x-component to match the price range.
    """
    scaled_lorenz = (lorenz_x - np.min(lorenz_x)) / (np.max(lorenz_x) - np.min(lorenz_x))
    scaled_lorenz = scaled_lorenz * (np.max(price_paths) - np.min(price_paths)) + np.min(price_paths)
    return scaled_lorenz

# Initialize Dash app
app = Dash(__name__)

# Layout of the dashboard
app.layout = html.Div([
    html.H1(f"Hybrid Monte Carlo-Chaos Attractor Dashboard ({symbol})"),
    dcc.Graph(id='live-graph'),
    dcc.Interval(id='interval-component', interval=60*1000, n_intervals=0),  # Update every 60 seconds
    html.Div(id='suggestions')
])

# Global variable to store the latest data
latest_data = {'price': None, 'paths': None, 'scaled_lorenz': None}

# Function to fetch live data and update the model
def update_data():
    while True:
        try:
            # Fetch live price data from Binance
            live_price = float(client.get_symbol_ticker(symbol=symbol)['price'])
            print(f"Live price of {symbol}: {live_price}")

            # Generate Monte Carlo price paths
            price_paths = monte_carlo_simulation(live_price)

            # Scale the Lorenz attractor's x-component
            scaled_lorenz = scale_lorenz(lorenz_x, price_paths)

            # Update the global variable
            latest_data['price'] = live_price
            latest_data['paths'] = price_paths
            latest_data['scaled_lorenz'] = scaled_lorenz

        except Exception as e:
            print(f"Error: {e}")

        # Wait for 60 seconds before the next update
        time.sleep(60)

# Start the data update thread
thread = threading.Thread(target=update_data)
thread.daemon = True
thread.start()

# Callback to update the graph and suggestions
@app.callback(
    [Output('live-graph', 'figure'),
     Output('suggestions', 'children')],
    [Input('interval-component', 'n_intervals')]
)
def update_graph(n):
    # Get the latest data
    live_price = latest_data['price']
    price_paths = latest_data['paths']
    scaled_lorenz = latest_data['scaled_lorenz']

    if live_price is None or price_paths is None or scaled_lorenz is None:
        return go.Figure(), "Waiting for data..."

    # Combine Monte Carlo paths with chaos attractor dynamics
    hybrid_paths = price_paths * scaled_lorenz[:, np.newaxis]

    # Create the graph
    fig = go.Figure()
    for i in range(hybrid_paths.shape[1]):
        fig.add_trace(go.Scatter(
            y=hybrid_paths[:, i],
            mode='lines',
            line=dict(width=1),
            name=f'Path {i+1}'
        ))
    fig.update_layout(
        title=f"Hybrid Monte Carlo-Chaos Attractor Price Paths (Live Data - {symbol})",
        xaxis_title="Time (Days)",
        yaxis_title="Price"
    )

    # Generate suggestions based on the latest price
    if live_price > np.mean(hybrid_paths[-1]):
        suggestion = "The current price is above the average predicted price. Consider taking profits or reducing exposure."
    else:
        suggestion = "The current price is below the average predicted price. Consider buying or increasing exposure."

    return fig, html.Div([
        html.H3("Trading Suggestion:"),
        html.P(suggestion)
    ])

# Run the Dash app
if __name__ == '__main__':
    app.run(debug=True)  # Use app.run instead of app.run_server

In [28]:
from binance.client import Client
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('BINANCE_API_KEY')
api_secret = os.getenv('BINANCE_SECRET_KEY')
client = Client(api_key, api_secret)

# Fetch live price data
symbol = 'SOLUSDT'
live_price = float(client.get_symbol_ticker(symbol=symbol)['price'])
print(f"Live price of {symbol}: {live_price}")

Live price of SOLUSDT: 131.02


In [29]:
# Import required libraries
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objs as go
from binance.client import Client
from dotenv import load_dotenv
import os
import numpy as np
from scipy.integrate import solve_ivp
import time
import threading

# Load environment variables from .env file
load_dotenv()

# Initialize Binance client
api_key = os.getenv('BINANCE_API_KEY')
api_secret = os.getenv('BINANCE_SECRET_KEY')
client = Client(api_key, api_secret)

# Define the symbol for live data (e.g., SOLUSDT)
symbol = 'SOLUSDT'

# Monte Carlo simulation function
def monte_carlo_simulation(live_price, n_simulations=1000, n_days=252, mu=0.0001, sigma=0.01):
    """
    Generate Monte Carlo price paths using Geometric Brownian Motion (GBM).
    """
    dt = 1 / n_days
    paths = np.zeros((n_days, n_simulations))
    paths[0] = live_price

    for t in range(1, n_days):
        rand = np.random.normal(0, 1, n_simulations)
        paths[t] = paths[t - 1] * np.exp((mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * rand)

    return paths

# Lorenz attractor function
def lorenz_system(t, state, sigma, rho, beta):
    """
    Define the Lorenz system of differential equations.
    """
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x * (rho - z) - y
    dzdt = x * y - beta * z
    return [dxdt, dydt, dzdt]

# Parameters for the Lorenz system
sigma = 10
rho = 28
beta = 8 / 3

# Initial conditions for the Lorenz system
initial_state = [1, 1, 1]

# Time span for solving the Lorenz system
t_span = (0, 50)
t_eval = np.linspace(t_span[0], t_span[1], 10000)

# Solve the Lorenz system
solution = solve_ivp(lorenz_system, t_span, initial_state, args=(sigma, rho, beta), t_eval=t_eval)

# Extract the x-component of the Lorenz attractor
lorenz_x = solution.y[0]

# Scale the Lorenz x-component to match the price range
def scale_lorenz(lorenz_x, price_paths):
    """
    Scale the Lorenz attractor's x-component to match the price range.
    """
    scaled_lorenz = (lorenz_x - np.min(lorenz_x)) / (np.max(lorenz_x) - np.min(lorenz_x))
    scaled_lorenz = scaled_lorenz * (np.max(price_paths) - np.min(price_paths)) + np.min(price_paths)
    return scaled_lorenz

# Initialize Dash app
app = Dash(__name__)

# Layout of the dashboard
app.layout = html.Div([
    html.H1(f"Hybrid Monte Carlo-Chaos Attractor Dashboard ({symbol})"),
    dcc.Graph(id='live-graph'),
    dcc.Interval(id='interval-component', interval=60*1000, n_intervals=0),  # Update every 60 seconds
    html.Div(id='suggestions')
])

# Global variable to store the latest data
latest_data = {'price': None, 'paths': None, 'scaled_lorenz': None}

# Function to fetch live data and update the model
def update_data():
    while True:
        try:
            # Fetch live price data from Binance
            live_price = float(client.get_symbol_ticker(symbol=symbol)['price'])
            print(f"Live price of {symbol}: {live_price}")

            # Generate Monte Carlo price paths
            price_paths = monte_carlo_simulation(live_price)

            # Scale the Lorenz attractor's x-component
            scaled_lorenz = scale_lorenz(lorenz_x, price_paths)

            # Update the global variable
            latest_data['price'] = live_price
            latest_data['paths'] = price_paths
            latest_data['scaled_lorenz'] = scaled_lorenz

        except Exception as e:
            print(f"Error: {e}")

        # Wait for 60 seconds before the next update
        time.sleep(60)

# Start the data update thread
thread = threading.Thread(target=update_data)
thread.daemon = True
thread.start()

# Callback to update the graph and suggestions
@app.callback(
    [Output('live-graph', 'figure'),
     Output('suggestions', 'children')],
    [Input('interval-component', 'n_intervals')]
)
def update_graph(n):
    # Get the latest data
    live_price = latest_data['price']
    price_paths = latest_data['paths']
    scaled_lorenz = latest_data['scaled_lorenz']

    if live_price is None or price_paths is None or scaled_lorenz is None:
        return go.Figure(), "Waiting for data..."

    # Combine Monte Carlo paths with chaos attractor dynamics
    hybrid_paths = price_paths * scaled_lorenz[:, np.newaxis]

    # Create the graph
    fig = go.Figure()
    for i in range(hybrid_paths.shape[1]):
        fig.add_trace(go.Scatter(
            y=hybrid_paths[:, i],
            mode='lines',
            line=dict(width=1),
            name=f'Path {i+1}'
        ))
    fig.update_layout(
        title=f"Hybrid Monte Carlo-Chaos Attractor Price Paths (Live Data - {symbol})",
        xaxis_title="Time (Days)",
        yaxis_title="Price"
    )

    # Generate suggestions based on the latest price
    if live_price > np.mean(hybrid_paths[-1]):
        suggestion = "The current price is above the average predicted price. Consider taking profits or reducing exposure."
    else:
        suggestion = "The current price is below the average predicted price. Consider buying or increasing exposure."

    return fig, html.Div([
        html.H3("Trading Suggestion:"),
        html.P(suggestion)
    ])

# Run the Dash app
if __name__ == '__main__':
    app.run(debug=True)  # Use app.run instead of app.run_server

Live price of SOLUSDT: 130.95


In [30]:
# Import required libraries
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objs as go
from binance.client import Client
from dotenv import load_dotenv
import os
import numpy as np
from scipy.integrate import solve_ivp
import time
import threading
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)

# Load environment variables from .env file
load_dotenv()

# Initialize Binance client
api_key = os.getenv('BINANCE_API_KEY')
api_secret = os.getenv('BINANCE_SECRET_KEY')
client = Client(api_key, api_secret)

# Define the symbol for live data (e.g., SOLUSDT)
symbol = 'SOLUSDT'

# Monte Carlo simulation parameters
n_simulations = 1000
n_days = 252  # Ensure this matches the Lorenz system's time span

# Monte Carlo simulation function
def monte_carlo_simulation(live_price, mu=0.0001, sigma=0.01):
    """
    Generate Monte Carlo price paths using Geometric Brownian Motion (GBM).
    """
    dt = 1 / n_days
    paths = np.zeros((n_days, n_simulations))
    paths[0] = live_price

    for t in range(1, n_days):
        rand = np.random.normal(0, 1, n_simulations)
        paths[t] = paths[t - 1] * np.exp((mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * rand)

    return paths

# Lorenz attractor function
def lorenz_system(t, state, sigma, rho, beta):
    """
    Define the Lorenz system of differential equations.
    """
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x * (rho - z) - y
    dzdt = x * y - beta * z
    return [dxdt, dydt, dzdt]

# Parameters for the Lorenz system
sigma = 10
rho = 28
beta = 8 / 3

# Solve the Lorenz system for the same duration as Monte Carlo simulation
t_span = (0, n_days)  # Match the number of days
t_eval = np.linspace(t_span[0], t_span[1], n_days)  # One point per day

# Solve the Lorenz system
solution = solve_ivp(lorenz_system, t_span, [1, 1, 1], args=(sigma, rho, beta), t_eval=t_eval)

# Extract the x-component of the Lorenz attractor
lorenz_x = solution.y[0]

# Scale the Lorenz x-component to match the price range
def scale_lorenz(lorenz_x, price_paths):
    """
    Scale the Lorenz attractor's x-component to match the price range.
    """
    scaled_lorenz = (lorenz_x - np.min(lorenz_x)) / (np.max(lorenz_x) - np.min(lorenz_x))
    scaled_lorenz = scaled_lorenz * (np.max(price_paths) - np.min(price_paths)) + np.min(price_paths)
    return scaled_lorenz

# Initialize Dash app
app = Dash(__name__)

# Layout of the dashboard
app.layout = html.Div([
    html.H1(f"Hybrid Monte Carlo-Chaos Attractor Dashboard ({symbol})"),
    dcc.Graph(id='live-graph'),
    dcc.Interval(id='interval-component', interval=60*1000, n_intervals=0),  # Update every 60 seconds
    html.Div(id='suggestions')
])

# Global variable to store the latest data (with thread lock)
import threading
latest_data = {'price': None, 'paths': None, 'scaled_lorenz': None}
data_lock = threading.Lock()

# Function to fetch live data and update the model
def update_data():
    while True:
        try:
            # Fetch live price data from Binance
            live_price = float(client.get_symbol_ticker(symbol=symbol)['price'])
            logging.info(f"Live price of {symbol}: {live_price}")

            # Generate Monte Carlo price paths
            price_paths = monte_carlo_simulation(live_price)

            # Scale the Lorenz attractor's x-component
            scaled_lorenz = scale_lorenz(lorenz_x, price_paths)

            # Update the global variable with thread lock
            with data_lock:
                latest_data['price'] = live_price
                latest_data['paths'] = price_paths
                latest_data['scaled_lorenz'] = scaled_lorenz

        except Exception as e:
            logging.error(f"Error in update_data: {e}")

        # Wait for 60 seconds before the next update
        time.sleep(60)

# Start the data update thread
thread = threading.Thread(target=update_data)
thread.daemon = True
thread.start()

# Callback to update the graph and suggestions
@app.callback(
    [Output('live-graph', 'figure'),
     Output('suggestions', 'children')],
    [Input('interval-component', 'n_intervals')]
)
def update_graph(n):
    # Get the latest data with thread lock
    with data_lock:
        live_price = latest_data['price']
        price_paths = latest_data['paths']
        scaled_lorenz = latest_data['scaled_lorenz']

    if live_price is None or price_paths is None or scaled_lorenz is None:
        return go.Figure(), "Waiting for data..."

    # Validate data shapes
    if price_paths.shape[0] != len(scaled_lorenz):
        return go.Figure(), "Data shape mismatch. Restart the app."

    # Combine Monte Carlo paths with chaos attractor dynamics
    hybrid_paths = price_paths * scaled_lorenz[:, np.newaxis]

    # Create the graph
    fig = go.Figure()
    for i in range(hybrid_paths.shape[1]):
        fig.add_trace(go.Scatter(
            y=hybrid_paths[:, i],
            mode='lines',
            line=dict(width=1),
            name=f'Path {i+1}'
        ))
    fig.update_layout(
        title=f"Hybrid Monte Carlo-Chaos Attractor Price Paths (Live Data - {symbol})",
        xaxis_title="Time (Days)",
        yaxis_title="Price"
    )

    # Generate suggestions based on the latest price
    if live_price > np.mean(hybrid_paths[-1]):
        suggestion = "The current price is above the average predicted price. Consider taking profits or reducing exposure."
    else:
        suggestion = "The current price is below the average predicted price. Consider buying or increasing exposure."

    return fig, html.Div([
        html.H3("Trading Suggestion:"),
        html.P(suggestion)
    ])

# Run the Dash app
if __name__ == '__main__':
    app.run(debug=False)  # Disable debug mode for stability

INFO:root:Live price of SOLUSDT: 130.73


In [31]:
# Import required libraries
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objs as go
from binance.client import Client
from dotenv import load_dotenv
import os
import numpy as np
from scipy.integrate import solve_ivp
import time
import threading
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)

# Load environment variables from .env file
load_dotenv()

# Initialize Binance client
api_key = os.getenv('BINANCE_API_KEY')
api_secret = os.getenv('BINANCE_SECRET_KEY')
client = Client(api_key, api_secret)

# Define the symbol for live data (e.g., SOLUSDT)
symbol = 'SOLUSDT'

# Monte Carlo simulation parameters
n_simulations = 1000
n_days = 252  # 1 year of trading days

def get_live_volatility(symbol, days=30):
    """Calculate live volatility using historical price data"""
    try:
        # Fetch historical price data (last 30 days with 1-hour candles)
        klines = client.get_historical_klines(
            symbol=symbol,
            interval=Client.KLINE_INTERVAL_1HOUR,
            start_str=f"{days} days ago UTC"
        )

        # Extract closing prices
        closes = np.array([float(entry[4]) for entry in klines])

        # Calculate logarithmic returns
        returns = np.diff(np.log(closes))

        # Compute volatility (annualized)
        volatility = np.std(returns) * np.sqrt(24 * 365)  # Hourly data to annualized

        return volatility

    except Exception as e:
        logging.error(f"Error calculating volatility: {e}")
        return 0.2  # Fallback value

# Lorenz attractor system (precomputed)
def lorenz_system(t, state, sigma, rho, beta):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x * (rho - z) - y
    dzdt = x * y - beta * z
    return [dxdt, dydt, dzdt]

# Precompute Lorenz attractor data
sigma = 10
rho = 28
beta = 8/3
t_span = (0, n_days)
t_eval = np.linspace(t_span[0], t_span[1], n_days)
solution = solve_ivp(lorenz_system, t_span, [1, 1, 1], args=(sigma, rho, beta), t_eval=t_eval)
lorenz_x = solution.y[0]

def monte_carlo_simulation(live_price, volatility, n_days=n_days):
    """Generate Monte Carlo paths with live volatility"""
    dt = 1/n_days
    paths = np.zeros((n_days, n_simulations))
    paths[0] = live_price

    for t in range(1, n_days):
        rand = np.random.normal(0, 1, n_simulations)
        paths[t] = paths[t-1] * np.exp((0 - 0.5*volatility**2)*dt + volatility*np.sqrt(dt)*rand)

    return paths

# Initialize Dash app
app = Dash(__name__)

# Layout of the dashboard
app.layout = html.Div([
    html.H1(f"Live Volatility-Adjusted Price Model: {symbol}"),
    dcc.Graph(id='live-graph'),
    dcc.Interval(id='interval-component', interval=60*1000, n_intervals=0),
    html.Div(id='volatility-display'),
    html.Div(id='suggestions')
])

# Shared data storage with thread lock
latest_data = {
    'price': None,
    'paths': None,
    'volatility': None,
    'lorenz_component': None
}
data_lock = threading.Lock()

def update_data():
    """Thread function to update live data"""
    while True:
        try:
            # Get live price
            live_price = float(client.get_symbol_ticker(symbol=symbol)['price'])

            # Calculate live volatility
            volatility = get_live_volatility(symbol)

            # Generate Monte Carlo paths with live volatility
            price_paths = monte_carlo_simulation(live_price, volatility)

            # Scale Lorenz component based on current volatility
            scaled_lorenz = (lorenz_x - np.mean(lorenz_x)) * (volatility * 0.5)

            with data_lock:
                latest_data.update({
                    'price': live_price,
                    'paths': price_paths,
                    'volatility': volatility,
                    'lorenz_component': scaled_lorenz
                })

            logging.info(f"Updated data - Price: {live_price}, Volatility: {volatility:.2%}")

        except Exception as e:
            logging.error(f"Update error: {e}")

        time.sleep(60)

# Start data thread
thread = threading.Thread(target=update_data)
thread.daemon = True
thread.start()

@app.callback(
    [Output('live-graph', 'figure'),
     Output('volatility-display', 'children'),
     Output('suggestions', 'children')],
    [Input('interval-component', 'n_intervals')]
)
def update_graph(n):
    """Update dashboard components"""
    with data_lock:
        data = latest_data.copy()

    if None in data.values():
        return go.Figure(), "Loading initial data...", ""

    try:
        # Combine Monte Carlo paths with chaos component
        hybrid_paths = data['paths'] + data['lorenz_component'][:, np.newaxis]

        # Create figure
        fig = go.Figure()
        for i in range(min(50, n_simulations)):  # Plot first 50 paths for clarity
            fig.add_trace(go.Scatter(
                y=hybrid_paths[:, i],
                mode='lines',
                line=dict(width=1),
                name=f'Path {i+1}'
            ))

        # Add current price line
        fig.add_hline(y=data['price'], line_dash="dot",
                     annotation_text="Current Price",
                     line_color="red")

        fig.update_layout(
            title=f"Volatility-Adjusted Price Paths ({symbol})",
            xaxis_title="Trading Days Ahead",
            yaxis_title="Price (USD)",
            showlegend=False
        )

        # Generate suggestions
        mean_prediction = np.mean(hybrid_paths[-1])
        suggestion = ("Consider taking profits" if data['price'] > mean_prediction
                      else "Consider accumulating") + f" (Current: ${data['price']:.2f}, Predicted: ${mean_prediction:.2f})"

        volatility_display = html.Div([
            html.H3("Market Conditions:"),
            html.P(f"Live Volatility: {data['volatility']:.2%} (annualized)"),
            html.P(f"Prediction Horizon: {n_days} trading days")
        ])

        return fig, volatility_display, html.Div([
            html.H3("Trading Suggestion:"),
            html.P(suggestion)
        ])

    except Exception as e:
        logging.error(f"Graph error: {e}")
        return go.Figure(), "Error generating graph", ""

if __name__ == '__main__':
    app.run(debug=False, port=8050)

In [36]:
# Import required libraries
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objs as go
from binance.client import Client
from dotenv import load_dotenv
import os
import numpy as np
from scipy.integrate import solve_ivp
from scipy.fft import fft, ifft
import time
import threading
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)

# Load environment variables
load_dotenv()

# Initialize Binance client
client = Client(os.getenv('BINANCE_API_KEY'), os.getenv('BINANCE_SECRET_KEY'))

# Configuration
SYMBOL = 'SOLUSDT'
N_SIMULATIONS = 500
N_DAYS = 252
CHAOS_MULTIPLIER = 0.3
DOMINANT_FREQUENCIES = 5

# Precompute Lorenz attractor
def lorenz_system(t, state, sigma=10, rho=28, beta=8/3):
    x, y, z = state
    return [sigma*(y-x), x*(rho-z)-y, x*y-beta*z]

t_eval = np.linspace(0, N_DAYS, N_DAYS)
solution = solve_ivp(lorenz_system, [0, N_DAYS], [1, 1, 1], t_eval=t_eval)
lorenz_x = solution.y[0]

def get_volatility(symbol, hours=24*14):
    """Get 14-day realized volatility"""
    klines = client.get_historical_klines(symbol, Client.KLINE_INTERVAL_1HOUR, f"{hours//24} days ago UTC")
    closes = np.array([float(k[4]) for k in klines])
    returns = np.diff(np.log(closes))
    return np.std(returns) * np.sqrt(365*24)

def monte_carlo_paths(price, volatility, n_days=N_DAYS):
    dt = 1/n_days
    paths = np.zeros((n_days, N_SIMULATIONS))
    paths[0] = price
    for t in range(1, n_days):
        paths[t] = paths[t-1] * np.exp(-0.5*volatility**2*dt + volatility*np.sqrt(dt)*np.random.normal(0,1,N_SIMULATIONS))
    return paths

def fourier_analysis(paths, n_components=DOMINANT_FREQUENCIES):
    """Extract dominant frequency components"""
    fpaths = fft(paths, axis=0)
    magnitudes = np.abs(fpaths)
    sorted_indices = np.argsort(magnitudes, axis=0)[-n_components:]

    filtered = np.zeros_like(fpaths)
    for i in range(N_SIMULATIONS):
        filtered[sorted_indices[:,i],i] = fpaths[sorted_indices[:,i],i]

    return np.real(ifft(filtered, axis=0))

def phase_aligned_average(paths):
    """Create phase-coherent average"""
    fpaths = fft(paths, axis=0)
    avg_phase = np.angle(np.mean(fpaths[1:DOMINANT_FREQUENCIES+1], axis=1))
    aligned = fpaths.copy()
    for i in range(1, DOMINANT_FREQUENCIES+1):
        aligned[i] = np.abs(fpaths[i]) * np.exp(1j*avg_phase[i-1])
    return np.real(ifft(aligned, axis=0))

# Dash App
app = Dash(__name__)

app.layout = html.Div([
    html.H1(f"Chaotic Market Analyzer: {SYMBOL}"),
    dcc.Graph(id='live-graph'),
    dcc.Interval(id='refresh', interval=60*1000),
    html.Div(id='stats')
])

# Shared data with thread safety
data_store = {
    'raw_paths': None,
    'filtered_paths': None,
    'consensus': None,
    'price': None,
    'volatility': None
}
lock = threading.Lock()

def data_thread():
    """Independent data processing thread"""
    while True:
        try:
            price = float(client.get_symbol_ticker(symbol=SYMBOL)['price'])
            vol = get_volatility(SYMBOL)

            # Generate base paths
            mc_paths = monte_carlo_paths(price, vol)

            # Add chaotic component
            chaos = (lorenz_x - lorenz_x.mean()) * vol * CHAOS_MULTIPLIER
            hybrid_paths = mc_paths + chaos[:,np.newaxis]

            # Signal processing
            filtered = fourier_analysis(hybrid_paths)
            consensus = phase_aligned_average(filtered)

            with lock:
                data_store.update({
                    'raw_paths': hybrid_paths,
                    'filtered_paths': filtered,
                    'consensus': consensus.mean(axis=1),
                    'price': price,
                    'volatility': vol
                })

            logging.info(f"Updated: ${price:.2f} Vol: {vol:.2%}")

        except Exception as e:
            logging.error(f"Data error: {e}")

        time.sleep(60)

threading.Thread(target=data_thread, daemon=True).start()

@app.callback(
    [Output('live-graph', 'figure'),
     Output('stats', 'children')],
    [Input('refresh', 'n_intervals')]
)
def update(_):
    with lock:
        data = {k:v.copy() if hasattr(v, 'copy') else v for k,v in data_store.items()}

    if data['price'] is None:
        return go.Figure(), "Initializing..."

    # Create visualization
    fig = go.Figure()

    # Plot filtered paths
    for i in range(min(50, N_SIMULATIONS)):
        fig.add_trace(go.Scatter(
            y=data['filtered_paths'][:,i],
            line=dict(width=1, color='rgba(100,100,100,0.1)'),
            showlegend=False
        ))

    # Plot consensus path
    fig.add_trace(go.Scatter(
        y=data['consensus'],
        line=dict(width=3, color='red'),
        name='Phase-Aligned Consensus'
    ))

    # Current price markers
    fig.add_hline(y=data['price'], line_dash="dot",
                 annotation_text="Current Price",
                 line_color="blue")

    fig.update_layout(
        title=f"Dominant Market Path Analysis (Volatility: {data['volatility']:.2%})",
        xaxis_title="Trading Days Horizon",
        yaxis_title="Price (USD)",
        hovermode="x unified"
    )

    stats = html.Div([
        html.H3("Key Statistics:"),
        html.P(f"Current Price: ${data['price']:.2f}"),
        html.P(f"30D Volatility: {data['volatility']:.2%}"),
        html.P(f"1Y Consensus Forecast: ${data['consensus'][-1]:.2f} (±{data['consensus'].std():.2f})")
    ])

    return fig, stats

if __name__ == '__main__':
    app.run(debug=False, port=8050)

In [34]:
# Import required libraries
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objs as go
from binance.client import Client
from dotenv import load_dotenv
import os
import numpy as np
from scipy.integrate import solve_ivp
from scipy.fft import fft, fftfreq
import time
import threading
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)

# Load environment variables
load_dotenv()

# Initialize Binance client
client = Client(os.getenv('BINANCE_API_KEY'), os.getenv('BINANCE_SECRET_KEY'))

# Configuration
SYMBOL = 'SOLUSDT'
N_SIMULATIONS = 500
N_DAYS = 252
SAMPLE_RATE = 1  # 1/day sampling

# Shared data storage with thread lock
data_store = {
    'fft_values': None,
    'frequencies': None,
    'consensus_fft': None,
    'price': None,
    'volatility': None
}
lock = threading.Lock()

# Precompute Lorenz attractor
def lorenz_system(t, state, sigma=10, rho=28, beta=8/3):
    x, y, z = state
    return [sigma*(y-x), x*(rho-z)-y, x*y-beta*z]

t_eval = np.linspace(0, N_DAYS, N_DAYS)
solution = solve_ivp(lorenz_system, [0, N_DAYS], [1, 1, 1], t_eval=t_eval)
lorenz_x = solution.y[0]

def get_volatility(symbol):
    """Get 14-day realized volatility"""
    klines = client.get_historical_klines(symbol, Client.KLINE_INTERVAL_1HOUR, "14 days ago UTC")
    closes = np.array([float(k[4]) for k in klines])
    returns = np.diff(np.log(closes))
    return np.std(returns) * np.sqrt(365*24)

def monte_carlo_paths(price, volatility):
    dt = 1/N_DAYS
    paths = np.zeros((N_DAYS, N_SIMULATIONS))
    paths[0] = price
    for t in range(1, N_DAYS):
        paths[t] = paths[t-1] * np.exp(-0.5*volatility**2*dt +
                                     volatility*np.sqrt(dt)*np.random.normal(0,1,N_SIMULATIONS))
    return paths

def calculate_fft_analysis(paths):
    """Perform FFT analysis on price paths"""
    ffts = fft(paths, axis=0)
    freqs = fftfreq(N_DAYS, 1/SAMPLE_RATE)
    return ffts, freqs

# Data processing thread
def data_thread():
    while True:
        try:
            price = float(client.get_symbol_ticker(symbol=SYMBOL)['price'])
            vol = get_volatility(SYMBOL)

            # Generate paths
            paths = monte_carlo_paths(price, vol)
            paths += (lorenz_x - lorenz_x.mean())[:, np.newaxis] * vol * 0.3

            # Calculate FFT
            ffts, freqs = calculate_fft_analysis(paths)

            with lock:
                data_store.update({
                    'fft_values': ffts,
                    'frequencies': freqs,
                    'consensus_fft': np.mean(np.abs(ffts), axis=1),
                    'price': price,
                    'volatility': vol
                })

            logging.info(f"Data updated - Price: {price:.2f}, Vol: {vol:.2%}")

        except Exception as e:
            logging.error(f"Data error: {e}")

        time.sleep(60)

# Start data thread
threading.Thread(target=data_thread, daemon=True).start()

# Create main dashboard (Port 8050)
app_main = Dash(__name__)
app_main.layout = html.Div([
    html.H1(f"Price Simulation: {SYMBOL}"),
    dcc.Graph(id='main-graph'),
    dcc.Interval(id='refresh-main', interval=60*1000)
])

@app_main.callback(
    Output('main-graph', 'figure'),
    [Input('refresh-main', 'n_intervals')]
)
def update_main(_):
    with lock:
        data = {k:v.copy() if hasattr(v, 'copy') else v for k,v in data_store.items()}

    if data['price'] is None:
        return go.Figure()

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        y=np.abs(data['consensus_fft']),
        mode='lines',
        name='Consensus Frequency Spectrum'
    ))
    fig.update_layout(
        title=f"Dominant Market Frequencies (Volatility: {data['volatility']:.2%})",
        xaxis_title="Frequency (cycles/day)",
        yaxis_title="Amplitude",
        yaxis_type="log"
    )
    return fig

# Create FFT dashboard (Port 8051)
app_fft = Dash(__name__)
app_fft.layout = html.Div([
    html.H1(f"Frequency Domain Analysis: {SYMBOL}"),
    dcc.Graph(id='fft-graph'),
    dcc.Interval(id='refresh-fft', interval=60*1000)
])

@app_fft.callback(
    Output('fft-graph', 'figure'),
    [Input('refresh-fft', 'n_intervals')]
)
def update_fft(_):
    with lock:
        data = {k:v.copy() if hasattr(v, 'copy') else v for k,v in data_store.items()}

    if data['frequencies'] is None:
        return go.Figure()

    fig = go.Figure()

    # Plot individual path spectra
    for i in range(min(10, N_SIMULATIONS)):  # First 10 paths
        fig.add_trace(go.Scatter(
            x=data['frequencies'],
            y=np.abs(data['fft_values'][:,i]),
            mode='lines',
            opacity=0.3,
            name=f'Path {i+1}',
            line=dict(width=1)
        ))

    # Plot consensus spectrum
    fig.add_trace(go.Scatter(
        x=data['frequencies'],
        y=data['consensus_fft'],
        mode='lines',
        name='Consensus Spectrum',
        line=dict(color='red', width=3)
    ))

    fig.update_layout(
        title="Fourier Transform of Price Paths",
        xaxis_title="Frequency (cycles/day)",
        yaxis_title="Magnitude (log scale)",
        xaxis_range=[0, 0.5],  # Nyquist frequency
        yaxis_type="log",
        showlegend=False
    )

    return fig

def run_app(app, port):
    app.run(debug=False, port=port)

if __name__ == '__main__':
    # Start both dashboards in separate threads
    threading.Thread(target=run_app, args=(app_main, 8050)).start()
    threading.Thread(target=run_app, args=(app_fft, 8051)).start()

In [ ]:
#!/usr/bin/env python3
import os
import logging
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Options
from qiskit_ibm_runtime.estimator import Estimator
from qiskit.circuit.library import EfficientSU2
from binance.client import Client

# Configure logging
logging.basicConfig(
    filename='/var/log/qc_price.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

class BrisbanePricePredictor:
    def __init__(self):
        self.service = QiskitRuntimeService(channel="ibm_quantum")
        self.backend = self.service.backend("ibm_brisbane")
        self.binance = Client(os.getenv('BINANCE_KEY'), os.getenv('BINANCE_SECRET'))

        # Hardware-aware configuration
        self.options = Options(
            resilience_level=2,  # M3 mitigation
            optimization_level=3,
            execution={"shots": 1024},
            dynamical_decoupling=True
        )

    def _fetch_features(self):
        """Get normalized technical indicators"""
        ticks = self.binance.get_historical_klines(
            'SOLUSDT', Client.KLINE_INTERVAL_15MIN,
            (datetime.now() - timedelta(hours=24)).strftime('%d %b %Y %H:%M:%S')
        )
        df = pd.DataFrame(ticks, columns=['timestamp', 'open', 'high', 'low', 'close'])
        features = df['close'].pct_change().dropna().values[:20]  # Use 20 features
        return (features - np.mean(features)) / np.std(features)

    def _create_circuit(self):
        """127-qubit Brisbane-optimized circuit"""
        qc = EfficientSU2(20, reps=3, entanglement="circular")
        qc.measure_all()
        return qc

    def predict(self):
        """Execute prediction workflow on Brisbane"""
        try:
            # 1. Prepare inputs
            features = self._fetch_features()
            qc = self._create_circuit()

            # 2. Hardware-aware transpilation
            transpiled = transpile(
                qc,
                backend=self.backend,
                scheduling_method="alap",
                layout_method="sabre"
            )

            # 3. Submit to Brisbane with error mitigation
            with Session(backend=self.backend) as session:
                estimator = Estimator(options=self.options)
                job = estimator.run(
                    circuits=transpiled,
                    parameter_values=[features],
                    parameters=[[]]  # No parameters for this circuit
                )
                logging.info(f"Job ID: {job.job_id}")
                result = job.result()

            # 4. Process results
            current_price = float(ticks[-1][4])
            counts = result.get_counts()[0]
            return self._interpret_counts(counts, current_price)

        except Exception as e:
            logging.error(f"Brisbane execution failed: {str(e)}")
            return None

    def _interpret_counts(self, counts, price):
        """Convert quantum measurements to price prediction"""
        # Using top 3 qubits for prediction
        total = sum(counts.values())
        up_prob = sum(v for k,v in counts.items() if k[-3] == '1') / total
        down_prob = 1 - up_prob

        return {
            'current': price,
            'target': price * (1 + 0.03*up_prob - 0.02*down_prob),
            'confidence': abs(up_prob - down_prob)
        }

def main():
    predictor = BrisbanePricePredictor()
    prediction = predictor.predict()
    if prediction:
        logging.info(
            f"Predicted price: ${prediction['target']:.2f} "
            f"(Current: ${prediction['current']:.2f}, "
            f"Confidence: {prediction['confidence']:.1%})"
        )

if __name__ == "__main__":
    main()